In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats.mstats import winsorize
import numpy as np
from scipy.stats import norm
from scipy import stats
from statsmodels.formula.api import ols
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
import warnings
warnings.filterwarnings('ignore')

In [2]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

from sklearn.datasets import make_classification
from collections import Counter

from matplotlib import pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_csv('./dataset/윈저데이터셋.csv', encoding='CP949')
df_y = df['label']
df_1 = df[['[제조]자기자본배율',
 '[제조]1주당순이익',
 '[제조]매출액순이익률',
 '[제조]부가가치',
 '[제조]총자본증가율',
 '[제조]총자본투자효율',
 '[제조]유동비율',
 '[제조]비유동자산회전률',
 '[제조]매입채무회전률',
 '[제조]유형자산증가율','label']]

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df_1)

df_scaled = scaler.transform(df_1)

df_scaled_df = pd.DataFrame(df_scaled,columns = df_1.columns)

## MDA 데이터셋 

In [6]:
from sklearn.model_selection import train_test_split
MDA_X = df_scaled_df.iloc[:,:-1]
MDA_y = df_y

In [7]:
MDA_X_samp_1, MDA_y_samp_1 = RandomUnderSampler(random_state=0,sampling_strategy=0.1).fit_resample(MDA_X, MDA_y)

print('RS 적용 전 학습용 피처/레이블 데이터 세트: ', MDA_X.shape, MDA_y.shape)
print('RS 적용 후 학습용 피처/레이블 데이터 세트: ', MDA_X_samp_1.shape, MDA_y_samp_1.shape)
print('RS 적용 후 레이블 값 분포: \n', pd.Series(MDA_y_samp_1).value_counts())

RS 적용 전 학습용 피처/레이블 데이터 세트:  (10652, 10) (10652,)
RS 적용 후 학습용 피처/레이블 데이터 세트:  (1793, 10) (1793,)
RS 적용 후 레이블 값 분포: 
 0    1630
1     163
Name: label, dtype: int64


In [8]:
MDA_Under_samp = pd.concat([MDA_X_samp_1,MDA_y_samp_1], axis=1)
MDA_Under_samp

[제조]자기자본배율  [제조]1주당순이익  [제조]매출액순이익률  [제조]부가가치  [제조]총자본증가율  [제조]총자본투자효율  \
0       0.364327   -0.268370     0.568340  2.499486    0.082208     0.124169   
1       0.372378   -0.131817     0.284942  1.196291    0.042988     0.260256   
2      -0.927537   -0.268370    -4.391570 -1.110847    0.680942    -1.765992   
3       0.252307    0.488586     0.621168  0.941650    0.215724     0.680431   
4      -0.082687    0.000436     0.507633 -0.618378   -0.067580     0.066473   
...          ...         ...          ...       ...         ...          ...   
1788   -1.148362   -1.531755    -1.627341 -1.145277   -2.321910    -1.765992   
1789   -0.478738   -1.910233    -0.977071 -0.703650    0.269547    -0.972049   
1790    2.585154    0.001511     0.407524  1.016888   -0.307491    -0.299142   
1791   -0.969556   -1.840344    -0.600569 -1.145277   -2.061554    -1.765992   
1792   -1.023755   -1.039304    -0.844274 -0.957690   -0.874933    -1.350207   

      [제조]유동비율  [제조]비유동자산회전률  [제조]매입채무회전률  [제조]유형자산증가율  label  
0    -0.725350     -0.826928    -0.801640     0.356101      0  
1    -0.548393     -0.006370    -0.673000     0.241722      0  
2    -0.733482     -1.044134    -0.953492     1.432766      0  
3     1.150418     -0.302012     0.932574    -0.416896      0  
4     2.851670     -0.235643     1.467443    -0.133903      0  
...        ...           ...          ...          ...    ...  
1788 -0.891937     -0.277878    -0.803090     1.027876      1  
1789 -0.363549     -0.428716    -0.528885    -0.398907      1  
1790  2.404075      2.666476    -0.319000    -0.614509      1  
1791 -0.318131      0.784020    -0.221312    -1.185061      1  
1792 -0.771215     -0.549386    -0.807927    -0.450727      1  

[1793 rows x 11 columns]

In [16]:
MDA_Under_samp.to_csv('./dataset/MDA_Undersampling.csv',encoding='CP949', index=False)

In [10]:
MDA_y_samp_1

0       0
1       0
2       0
3       0
4       0
       ..
1788    1
1789    1
1790    1
1791    1
1792    1
Name: label, Length: 1793, dtype: int64

In [11]:
MDA_x_train, MDA_x_test, MDA_y_train, MDA_y_test = train_test_split(MDA_X_samp_1, MDA_y_samp_1, test_size=0.2, random_state=0)

In [12]:
MDA_X_samp_2, MDA_y_samp_2 = RandomOverSampler(random_state=0,sampling_strategy=0.25).fit_resample(MDA_X_samp_1, MDA_y_samp_1)

print('RS 적용 전 학습용 피처/레이블 데이터 세트: ', MDA_X_samp_1.shape, MDA_y_samp_1.shape)
print('RS 적용 후 학습용 피처/레이블 데이터 세트: ', MDA_X_samp_2.shape, MDA_y_samp_2.shape)
print('RS 적용 후 레이블 값 분포: \n', pd.Series(MDA_y_samp_2).value_counts())

RS 적용 전 학습용 피처/레이블 데이터 세트:  (1793, 10) (1793,)
RS 적용 후 학습용 피처/레이블 데이터 세트:  (2037, 10) (2037,)
RS 적용 후 레이블 값 분포: 
 0    1630
1     407
Name: label, dtype: int64


In [13]:
MDA_x_train2, MDA_x_test2, MDA_y_train2, MDA_y_test2 = train_test_split(MDA_X_samp_2, MDA_y_samp_2, test_size=0.2, random_state=0)

In [18]:
MDA_train = pd.concat([MDA_x_train2, MDA_y_train2], axis=1)
MDA_test = pd.concat([MDA_x_test2, MDA_y_test2], axis=1)
MDA_Under_Over_Sampling = pd.concat([MDA_X_samp_2,MDA_y_samp_2], axis=1)
MDA_train.to_csv('./dataset/MDA_trainUnderOverSampling.csv', encoding='CP949', index=False)
MDA_test.to_csv('./dataset/MDA_testUnderOverSampling.csv', encoding='CP949', index=False)